In [ ]:
import sys
sys.path.append("..")
import Data as dt
import Temp_lib as lib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
data = dt.load_dataList(ticker_list=['A'])
data = data['A'].loc['2012-01-01':'2015-01-01']
plt.figure(figsize=(17, 4))
plt.plot(data['close'])
plt.show()

In [ ]:
tripleBarrier_params = {
    "upper_barrier": [0.8],
    "lower_barrier": [1.5],
    "vertical_barrier": [20],
    "window": [20],
    "smoothing_method": [None],
    "window_smooth": [5],
    "lambda_smooth": [0.2],
}

close_series = data['close']

tripleBarrier_labeller = lib.tripleBarrier_labeller(data=close_series, params=tripleBarrier_params, n_jobs=1)
labels_df = tripleBarrier_labeller.extract()
data.loc[:, 'labels'] = labels_df['set_0']
lib.plot_price_with_labels(price_series=close_series, label_series=data['labels'])

In [ ]:
def mpNumCoEvents(closeIdx,t1,molecule):
    """
    Compute the number of concurrent events per bar.
    +molecule[0] is the date of the first event on which the weight will be computed
    +molecule[-1] is the date of the last event on which the weight will be computed
    Any event that starts before t1[molecule].max() impacts the count.
    """
    #1) find events that span the period [molecule[0],molecule[-1]]
    t1=t1.fillna(closeIdx[-1]) # unclosed events still must impact other weights
    t1=t1[t1>=molecule[0]] # events that end at or after molecule[0]
    t1=t1.loc[:t1[molecule].max()] # events that start at or before t1[molecule].max()
    #2) count events spanning a bar
    iloc=closeIdx.searchsorted(np.array([t1.index[0],t1.max()]))
    count=pd.Series(0,index=closeIdx[iloc[0]:iloc[1]+1])
    
    for tIn,tOut in t1:
        count.loc[tIn:tOut]+=1
    
    return count.loc[molecule[0]:t1[molecule]].max()

In [ ]:
labels_series = data['labels'].copy()
close_series = data['close'].copy()
window = 20
upper_barrier = 0.8
vertical_barrier = 20
#*________________________

# Drop NaN values from labels_series
labels = labels_series.dropna().copy()
close = close_series.loc[labels.index].copy()
vol = close_series.pct_change().rolling(window).std() * np.sqrt(window)
vol.rename('vol', inplace=True)

auxiliary = pd.concat([labels, close, vol], axis=1).dropna().reset_index()

for idx, row in auxiliary.iterrows():
    
    if row['labels'] == 1:
        current_close = row['close']
        barrier = row['vol'] * upper_barrier
        target_close = current_close * (1 + barrier)
        
        barrier_hit_idx = auxiliary[(auxiliary['close'] >= target_close) & (auxiliary.index > idx)].index.min()

    elif row['labels'] == -1:
        current_close = row['close']
        barrier = row['vol'] * upper_barrier
        target_close = current_close * (1 - barrier)
        
        barrier_hit_idx = auxiliary[(auxiliary['close'] <= target_close) & (auxiliary.index > idx)].index.min()
    else:
        barrier_hit_idx = vertical_barrier + idx

    auxiliary.loc[idx, 'start_event'] = idx
    auxiliary.loc[idx, 'end_event'] = barrier_hit_idx
    
auxiliary = auxiliary.drop(columns=['vol'])

auxiliary["concurrent_events"] = 0  # Initialize column

def count_concurrent_events(row, df):
    label = row['labels']
    start_idx = row['start_event']
    end_idx = row['end_event']
    
    mask_prev = (df['labels'] == label) & (df['start_event'] < start_idx) & (df['end_event'] > start_idx)
    mask_next = (df['labels'] == label) & (df['start_event'] < end_idx) & (df['end_event'] > end_idx)
    
    nb_concurrents_events_prev = df[mask_prev].shape[0]
    
    return nb_concurrents_events_prev if nb_concurrents_events_prev > 0 else 0

auxiliary['concurrent_events'] = auxiliary.apply(lambda row: count_concurrent_events(row, auxiliary), axis=1)

In [ ]:
auxiliary

In [ ]:
def label_uniqueness(labels_series: pd.Series, close_series: pd.Series):


    

In [ ]:
closeIdx = pd.Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
t1 = pd.Series({
    1: 4,  # Event starts at 1, ends at 4
    2: 5,  # Event starts at 2, ends at 5
    3: 6,  # Event starts at 3, ends at 6
    4: 7,  # Event starts at 4, ends at 7
    5: 8   # Event starts at 5, ends at 8
})
molecule = [2, 5]

# Call the function
result = mpNumCoEvents(closeIdx, t1, molecule)

# Print the result
print("Maximum number of concurrent events:", result)